In [1]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
    
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import pandas as pd
import orca
# import os; os.chdir('/home/juan/ual_model_workspace/spring-2019-models/')
import warnings; warnings.simplefilter('ignore')
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns

# from scripts import datasources, models, variables, utils

In [2]:
from scripts import datasources, models, variables, utils

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [3]:
import urbansim_templates
urbansim_templates.__version__

'0.1.1'

# Loading data

In [4]:
orca.run(['initialize_network_small', 'initialize_network_walk','impute_missing_skims']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Running step 'impute_missing_skims'
Time to execute step 'impute_missing_skims': 90.31 s
Total time to execute iteration 1 with iteration value None: 90.31 s


In [8]:
#Loading Data 
beam_skims = orca.get_table('beam_skims').to_frame()
students = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/students_with_school_id.csv')
schools = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/schools.csv')
parcels = orca.get_table('parcels').to_frame()

#Preprocessing
students = students[students.AGE <= 18]

In [11]:
#Adding list of grades offered by each school
list_grades = []
for index, row in schools.loc[:,schools.columns.str.startswith("grade_")].iterrows():
    x = np.array(row)
    list_grades.append(x)
    
schools['list_grades'] = list_grades

## Defining set choice contrains

In [12]:
#Small netwrok 
nodessmall = pd.read_csv('/home/data/fall_2018/bay_area_tertiary_strongly_nodes.csv').set_index('osmid')
edgessmall = pd.read_csv('/home/data/fall_2018/bay_area_tertiary_strongly_edges.csv')
netsmall = pdna.Network(nodessmall.x, nodessmall.y, edgessmall.u,
                                edgessmall.v, edgessmall[['length']],
                                twoway=False)
netsmall.precompute(25000)

def node_id_small(x, y, netsmall):
    """ Return the node ID given a pair of coordinates"""
    idssmall = netsmall.get_node_ids(x, y)
    return idssmall

# schools['nodeID'] = node_id_small(schools, netsmall)

In [13]:
#Merging students and schools 
students = students.merge(schools, how = 'left', on = 'school_id')

# Droping students with no assigned school
students = students.dropna(subset=['school_id'])

In [14]:
#Define the node ID for each home and school location in students dataset
students['node_id_home'] = node_id_small(students.HXCORD, students.HYCORD, netsmall)
students['node_id_school'] = node_id_small(students.Longitude, students.Latitude, netsmall)

#Creating a df for public shcools only
df_public = students[students.school_id <= 2827].loc[:,['SAMPN', 'PERNO', 'school_id',
                                                        'AGE','HCITY','HYCORD','HXCORD',
                                                        'SNAME_lookup','SCITY_lookup', 
                                                        'node_id_home', 'node_id_school']]

#Creating a df for private shcools only
df_private = students[students.school_id > 2827].loc[:,['SAMPN', 'PERNO', 'school_id',
                                                        'AGE','HCITY','HYCORD','HXCORD',
                                                        'SNAME_lookup','SCITY_lookup', 
                                                        'node_id_home', 'node_id_school']]

In [15]:
#Setting public and private schools as POIs. 
netsmall.set_pois('public_school', 500000, 10000, 
                  schools[schools.type == 'public'].Longitude, 
                  schools[schools.type == 'public'].Latitude, )

netsmall.set_pois('private_school', 500000, 10000, 
                  schools[schools.type == 'private'].Longitude, 
                  schools[schools.type == 'private'].Latitude)

In [16]:
#Public schools. 95% of the time, the school is within the x's closest schools. Finding x
n =50

# n closest public schools per each node in netsmall
distance_matrix_public = netsmall.nearest_pois(200000, 
                                               'public_school', 
                                               num_pois=n,
                                               include_poi_ids=True)

# Selects POIS id's only 
public_nodes = distance_matrix_public.iloc[:,n:]

# Creates a list of n closest POIs for each node
list_values = []
for index, row in public_nodes.iterrows():
    x = np.array(row)
    x = x[~np.isnan(x)]
    list_values.append(x)

#Add created list to public nodes 
public_nodes['list_values'] = list_values


merge = df_public.merge(public_nodes.loc[:,['list_values']], how = 'left', left_on = 'node_id_home', right_index= True)

school_position = []
for index, row in merge.iterrows():
    school_position.append(np.isin(row['school_id']-1000,row['list_values'])==True)
    
merge['school_position'] = school_position

merge.school_position.mean()

0.9131504922644164

In [17]:
#Private schools. 95% of the time, the school is within the x's closest schools. Finding x
n =100

distance_matrix_private = netsmall.nearest_pois(100000, 
                                                'private_school', 
                                                num_pois=n, 
                                                include_poi_ids=True)

private_nodes = distance_matrix_private.iloc[:,n:]

list_values = []
for index, row in private_nodes.iterrows():
    x = np.array(row)
    x = x[~np.isnan(x)]
    list_values.append(x)
    
private_nodes['list_values'] = list_values


merge = df_private.merge(private_nodes.loc[:,['list_values']], how = 'left', left_on = 'node_id_home', right_index= True)

school_position = []
for index, row in merge.iterrows():
    school_position.append(np.isin(row['school_id']-1000,row['list_values'])==True)
    
merge['school_position'] = school_position

merge.school_position.mean()

0.8758465011286681

In [32]:
# students = students[students.AGE <= 18]

In [18]:
def school_available(age, grades_offered):
    """ Checks if a school offers a grade according to the age range
    Input: Age: int in range (5-18)
           grades_offered: 13 element array
    Output: True if grade if offered acoording to age, false otherwise. 
     """
    index = age - 6
    
    if index < 0:
        index = 0

    if index < 12:
        result = (grades_offered[index] == 1) | (grades_offered[index + 1] == 1)
        
    elif index == 12:
        result = (grades_offered[index] == 1)
    
    return result

In [19]:
def school_choice_set(house_node_id, kid_age):
    """ Determines the school choice set given home node id and age of the student (4-18)
    Output: Pandas series with available school IDs"""
    
    public_id = distance_matrix_public.iloc[:,50:].loc[house_node_id] + 1000
    private_id = distance_matrix_private.iloc[:,100:].loc[house_node_id] + 1000
    schools_filter = pd.concat([public_id, private_id])
    schools_set = schools[schools.school_id.isin(schools_filter)]
    school_availability = [school_available(kid_age, x) for x in schools_set.list_grades]
    schools_available = schools_set[school_availability].school_id
    
    return schools_available

In [27]:
school_choice_set(5418694673, 5)


944     1944
946     1946
947     1947
948     1948
949     1949
950     1950
952     1952
961     1961
963     1963
972     1972
973     1973
1032    2032
1054    2054
1056    2056
1057    2057
1059    2059
1061    2061
1062    2062
1066    2066
1068    2068
1069    2069
1072    2072
1073    2073
1074    2074
1075    2075
1076    2076
1077    2077
1078    2078
1079    2079
1080    2080
        ... 
2289    3289
2290    3290
2291    3291
2292    3292
2293    3293
2294    3294
2295    3295
2296    3296
2297    3297
2298    3298
2299    3299
2300    3300
2302    3302
2306    3306
2311    3311
2316    3316
2324    3324
2329    3329
2337    3337
2342    3342
2349    3349
2359    3359
2360    3360
2362    3362
2363    3363
2365    3365
2373    3373
2429    3429
2455    3455
2465    3465
Name: school_id, Length: 109, dtype: int64

# Additional code

In [222]:
# #Creating school availability matrix based on distance
# school_availability_distance = pd.DataFrame(nodessmall.index)
# for x in range(1000,3547,1):
#     name = "school_" + str(x)
#     school_availability_distance[name] = x
# school_availability_distance = school_availability_distance.set_index('osmid')

In [159]:

a = students.sample(10)
a.loc[:,['AGE','HCITY','HYCORD','HXCORD', 'school_id']]

,AGE,HCITY,HYCORD,HXCORD,school_id
1399,8,SAN JOSE,37.380541,-121.881397,3377.0
1207,12,SAN MATEO,37.559456,-122.313309,3245.0
2214,12,SAN FRANCISCO,37.748253,-122.466109,NaN
2976,17,PALO ALTO,37.432141,-122.135012,2409.0
3383,16,SAN JOSE,37.293068,-121.758836,3352.0
2372,16,LOS ALTOS,37.355706,-122.063276,3364.0
3196,12,SAN JOSE,37.223990,-121.895572,2516.0
2784,13,VACAVILLE,38.361158,-121.950229,2611.0
3116,13,PIEDMONT,37.823476,-122.248137,1336.0
2263,15,DANVILLE,37.802477,-121.934218,1647.0


In [320]:
students[(students.SAMPN == 7176467) & (students.PERNO ==4) ][['HCITY', 'HYCORD','HXCORD',
                                                                'AGE','SNAME_lookup','SCITY_lookup', 'school_id']]

,HCITY,HYCORD,HXCORD,AGE,SNAME_lookup,SCITY_lookup,school_id
3107,OAKLAND,37.739792,-122.174284,17,skyline high,oakland,1271.0


In [321]:
schools[schools.school_id == 1271]

,CDSCode,School,District,County,Street,City,Zip,State,Latitude,Longitude,...,Total Enrollment,sw_rank_2108,sw_rank_2017,sw_rank_2016,ss_rank_2018_,ss_rank_2017,ss_rank_2016,nodeID,parcel_id,school_id
271,1612590137943,Skyline High,Oakland Unified,Alameda,12250 Skyline Boulevard,Oakland,94619-2425,CA,37.797495,-122.1636,...,1843.0,4.0,1.0,2.0,NaN,2.0,2.0,261597288,57510,1271


In [258]:
d[d.type == 'public'].sort_values('dist', ascending=False)

,SAMPN,PERNO,zone_id_home,school_id,zone_id,type,from_zone_id,to_zone_id,dist
629,1491094,6,1341,2264.0,653,public,1341,653,165166.5642
1767,1828414,3,1393,1430.0,1186,public,1393,1186,136053.6036
2375,1775595,3,535,1723.0,1446,public,535,1446,105315.2096
170,2990467,2,1250,2092.0,259,public,1250,259,100921.7114
1817,1811113,4,109,2435.0,541,public,109,541,78648.4458
2404,1788271,3,247,2631.0,1237,public,247,1237,77489.7210
1816,1781042,3,168,2435.0,541,public,168,541,77344.8804
1085,1390386,5,893,2462.0,694,public,893,694,76524.1170
1422,1440433,2,1081,1392.0,744,public,1081,744,68944.1256
1723,1892396,3,300,1044.0,1018,public,300,1018,68091.1754


In [ ]:
# os.listdir("/home/data/fall_2018/")

In [37]:
# # Estimating 95th percentail of distances between home and school
# a = schools.merge(parcels[['zone_id']], left_on ='parcel_id', right_index=True)
# b = students[['SAMPN','PERNO','zone_id_home', 'school_id']].merge(a[['zone_id','school_id','type']], on = 'school_id')
# c = beam_skims.reset_index(level=['from_zone_id', 'to_zone_id'])
# d = b.merge(c.loc[:,['from_zone_id','to_zone_id', 'dist']], 
#             left_on = ['zone_id_home','zone_id'], 
#             right_on = ['from_zone_id', 'to_zone_id'])

# public_dist_limit = d[d.type == 'public'].quantile(1).dist
# private_dist_limit = d[d.type == 'private'].quantile(1).dist

# sns.distplot(d[d.type == 'public'].dist,kde=False)
# sns.distplot(d[d.type == 'private'].dist,kde=False);
# d.sort_values('dist', ascending=False)

In [38]:
# a = students.loc[:,['HCITY','SNAME_lookup', 'SCITY_lookup', 'SZIP_lookup',
#                 'SXCORD_lookup', 'SYCORD_lookup', 'SPrimaryCity_lookup',]]
# same_city = []
# for index, row in a.iterrows():
#     if row['HCITY'].lower() == row['SCITY_lookup']:
#         same_city.append(True)
#     else:
#         same_city.append(False)

# a['same_city'] = same_city
# a.same_city.mean()

In [ ]:
#'network_aggregations_small'])
# orca.run(['initialize_network_walk', 'network_aggregations_walk'])

# orca.get_table('nodeswalk').to_frame().to_csv('./data/walk_net_vars.csv')
# orca.get_table('nodessmall').to_frame().to_csv('./data/drive_net_vars.csv')
# orca.get_table('zones').to_frame().to_csv('./data/zones_w_access_vars-2025-b-lt.csv')

In [7]:
# os.listdir("/home/data/spring_2019/2025")

In [ ]:
# a = students.loc[:,['SAMPN', 'PERNO', 'AGE', 'HPrimaryCity','school_id']].merge(schools.loc[:,['school_id','City','District']], how = 'inner', on = 'school_id')
# students[students.SAMPN == 1342402].loc[:,['SNAME_lookup','school_id', 'HPrimaryCity']]
# schools[schools.school_id == 1710]